In [ ]:
# Import the igraph library for graph-based analyses
import igraph as ig

In [ ]:
# Import scvelo, a key library for RNA velocity analysis
import scvelo as scv


In [ ]:
# Import matplotlib for plotting
import matplotlib.pyplot as plt




In [ ]:
# Check the version of scvelo to ensure compatibility

print(scv.__version__)


In [ ]:
# Import additional essential libraries for single-cell analysis and data manipulation
import scanpy as sc
import cellrank as cr
import numpy as np
import pandas as pd
import anndata as ad

In [ ]:
import loompy

In [ ]:
scv.settings.verbosity = 3
scv.settings.set_figure_params('scvelo', facecolor='white', dpi=100, frameon=False)
cr.settings.verbosity = 2

In [ ]:
adata = sc.read_h5ad('ALTH.combined_PRORE2.h5ad')

In [ ]:
print(adata)

In [ ]:
ldata1 = scv.read('New_Alloxan_possorted_genome_bam_CLF5P.loom', cache=True)
ldata2 = scv.read('THR_possorted_genome_bam_SI7SF.loom', cache=True)



In [ ]:
print(ldata1)

In [ ]:
print(ldata2)

In [ ]:
# Rename barcodes to ensure uniqueness and consistent format
barcodes1 = [f'Alloxan_{bc.split(":")[1]}-1_01' for bc in ldata1.obs.index.tolist()]
ldata1.obs.index = barcodes1

barcodes2 = [f'THR_{bc.split(":")[1]}-1_02' for bc in ldata2.obs.index.tolist()]
ldata2.obs.index = barcodes2


In [ ]:
# make variable names unique
ldata1.var_names_make_unique()
ldata2.var_names_make_unique()


In [ ]:
ldata = ldata1.concatenate([ldata2])

In [ ]:
ldata

In [ ]:
# Align variables (features) between adata and ldata_combined if necessary
common_genes = adata.var_names.intersection(ldata.var_names)
adata = adata[:, common_genes]
ldata = ldata[:, common_genes]

# Verify alignment
print(adata.shape)
print(ldata.shape)

In [ ]:
# merge matrices into the original adata object
adata = scv.utils.merge(adata, ldata)


In [ ]:
adata

In [ ]:
scv.pl.proportions(adata, groupby='CellType')

In [ ]:
scv.pl.proportions(adata, groupby='orig.ident')

In [ ]:
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=30, use_rep='X_pca')

In [ ]:
scv.tl.recover_dynamics(adata)


In [ ]:
scv.tl.velocity(adata, mode='dynamical')
scv.tl.velocity_graph(adata)

In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap', color=['seurat_clusters'], save='embedding_stream.pdf', title='')

In [ ]:
scv.pl.velocity_embedding_stream(
    adata,
    basis='umap',
    color=['seurat_clusters', 'orig.ident'],
    legend_loc='center right',
    save='embedding_stream.pdf',
    title='',
    n_neighbors=min(12, adata.n_obs)  # Adjust based on actual number of observations
)


In [ ]:
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=80)

In [ ]:
scv.tl.velocity_pseudotime(adata)
scv.pl.scatter(adata, color= 'latent_time', cmap='gnuplot')

In [ ]:

# Compute PAGA based on seurat_clusters

scv.tl.paga(adata, groups='seurat_clusters')
transitions_confidence = adata.uns['paga']['transitions_confidence'].copy()
cluster_latent_times = adata.obs.groupby('seurat_clusters')['latent_time'].mean()

for i, source_cluster in enumerate(cluster_latent_times.index):
    for j, target_cluster in enumerate(cluster_latent_times.index):
        if cluster_latent_times[source_cluster] < cluster_latent_times[target_cluster]:
            transitions_confidence[i, j] = 1
        else:
            transitions_confidence[i, j] = 0

adata.uns['paga']['transitions_confidence'] = transitions_confidence


In [ ]:
scv.pl.scatter(adata, basis='umap', color='seurat_clusters')
scv.pl.paga(adata, basis='umap', size=50, alpha=.1, edge_width_scale=.2, node_size_scale=1.5)
